In [ ]:
import torch
import random
import pandas as pd
import numpy as np
from IPython.display import clear_output
import torch
from torchvision import transforms
import warnings


# to ignore torchvision model warnings
warnings.filterwarnings(
    action='ignore',
    category=DeprecationWarning,
    
    module=r'.*randpool'
)

In [ ]:
# **Overview**
<font size="3">
This Notebook aims to demonstrate one of the inherent weaknesses of using convolutional neural networks (CNN's) in safety critical systems such as security systems, autonomous driving applications and more. The notebook offers a walk through of the steps and code necessary for you to understand the concepts involved and to try out the idea for yourself. Please feel free to re-use any code you see in this document.

* <pre> demo.ipynb   ->  demonstration notebook to follow along</pre> 
*  <pre> image.py     ->  supporting python file that provides the Image class and additional functionality I created</pre> 
</br>
</font>



## The weakness
<font size="3">**The Architecture:**
CNN's and NN's rely on precise weights that have been calibrated through the propagation of training errors to correct these prediction weights that are stored in the neurons of the hidden layers in the networks.</font>
</br></br>
<font size="3">**The Hypotheses:**
If we had access the the model (and therefore the neuron weights) used in these safety critical systems we could potential exploit their over reliance on particular neurons and their associated weights to adjust the model predictions to our benefit.</font>
</br></br>
<font size="3">**The Approach:**
This notebook uses the famous ResNet-18 model as an example of how these weights can be used to our advantage as well as some new arbitrary and unseen images for the model to make predictions on.</font>




In [ ]:
from PIL import Image

In [ ]:
<font size="3">We will be using the Python Imaging Library (PIL) to make precise changes to our images</font>

In [ ]:
our_banana_raw = Image.open('/Users/ted.taylor/Downloads/ban.jpg')
print(f"Original size : {our_banana_raw.size}") # 5464x3640

our_banana = our_banana_raw.resize((256, 256))

In [ ]:
<font size="3">Now we need to import an example image for the model to make predictions on, in this case our example image is just a banana</font>

In [ ]:
from image import Image, pred

# to ignore torchvision model warnings
warnings.filterwarnings(action='ignore',category=DeprecationWarning,module=r'.*randpool')

In [ ]:
<font size="3">We should also import our supporting python file  (and the pred method) which contains many of the supporting methods I created </font>

In [ ]:
our_image = Image(our_banana)
our_image.matrix

In [ ]:
<font size="3">We can then create an Image object by passing in our image we loaded, we can then view this image using the .matrix function on our Image object</font>


In [ ]:
print(f"{len(pred(our_image.matrix))}\n")
pred(our_image.matrix,5)

In [ ]:
<font size="3">By passing our image to the pred() function we can see it returns a list containing 1000 elements, we can also pass in an integer to limit the results to the top 'n' predictions, in this case we only really want to look at the top 5</font>
</br> </br> 
<font size="3">**As we can see the ResNet-18 model stored within the Image class is 99.9% sure our image is a banana!** </font>
</br> </br> 
<font size="3"> Lets aim to trick the model into thinking this banana is a 'golf ball' to the point where it is 99% confident</font>


In [ ]:
white_image = our_banana_raw.resize((256, 256))
for i in range(0,255):
    for j in range(0,255):
        white_image.load()[i,j] = (255,255,255)
black_image = our_banana_raw.resize((256, 256))
for i in range(0,255):
    for j in range(0,255):
        black_image.load()[i,j] = (0,0,0)

In [ ]:
white_image = our_banana_raw.resize((256, 256))
for i in range(0,255):
    for j in range(0,255):
        white_image.load()[i,j] = (255,255,255)
black_image = our_banana_raw.resize((256, 256))
for i in range(0,255):
    for j in range(0,255):
        black_image.load()[i,j] = (0,0,0)

In [5]:
def imageResize(image_param):
    
    from torchvision import transforms
    #
    # Create a preprocessing pipeline
    #
    preprocess = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )])
    #
    # Pass the image for preprocessing and the image preprocessed
    #
    img_cat_preprocessed = preprocess(image_param)
    return img_cat_preprocessed

In [6]:
def pred(image_param):
    img_cat_preprocessed = imageResize(image_param)
    #
    # Reshape, crop, and normalize the input tensor for feeding into network for evaluation
    #
    batch_img_cat_tensor = torch.unsqueeze(img_cat_preprocessed, 0)

    model.eval()
    #
    # Get the predictions of image as scores related to how the loaded image
    # matches with 1000 ImageNet classes. The variable, out is a vector of 1000 scores
    #
    out = model(batch_img_cat_tensor)

    # Load the file containing the 1,000 labels for the ImageNet dataset classes
    #
    with open('/Users/ted.taylor/Downloads/imagenet_classes.txt') as f:
        labels = [line.strip() for line in f.readlines()]
    #
    # Find the index (tensor) corresponding to the maximum score in the out tensor.
    # Torch.max function can be used to find the information
    #
    _, index = torch.max(out, 1)
    #
    # Find the score in terms of percentage by using torch.nn.functional.softmax function
    # which normalizes the output to range [0,1] and multiplying by 100
    #
    percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
    #
    # Print the name along with score of the object identified by the model
    #
    #print(labels[index[0]], percentage[index[0]].item())
    #
    # Print the top 5 scores along with the image label. Sort function is invoked on the torch to sort the scores.
    #
    _, indices = torch.sort(out, descending=True)
    preds = [(labels[idx], percentage[idx].item()) for idx in indices[0][:]]
    
    return (preds)

In [ ]:

result = trick(og_image.resize((256, 256)),'eggnog')

# Thoughts on improvements ?
<br>- Count what colors and sizes using in plots - once it gets to 25% restart with most common color-size weights increased proba
<br>- use k counter to nudge out of a local minimum - create thick white border around edge of image to nudge along
<br>- total pixel value count - if less than 1/4 of pixels != 255,255,255 at 25% restart process (avoids creating small objects/ noise)